In [3]:
#先import
import pandas as pd
from numpy import random
import math
import time

In [17]:
nweek = 10
weeks = range(1,nweek+1)
arrival_mean = {1:1000,2:1000,3:1000,4:1000,5:1000,6:1000,7:1000,8:1000,9:2000,10:2000}
arrival_sd = {1:250,2:250,3:250,4:250,5:250,6:250,7:250,8:250,9:500,10:500}
WTP = {1:1000,2:1000,3:1000,4:1000,5:1000,6:1000,7:590,8:590,9:590,10:590}
WTP_0 = {1:590,2:590,3:590,4:590,5:590,6:590,7:290,8:290,9:290,10:290}
interest_prob = 1/50
inventory = {}
revenue = {}
price = {}

In [5]:
#小函数模块
#生产感兴趣的人数
def interested (week):
    return int((max(0,random.normal(arrival_mean[week],arrival_sd[week])))*interest_prob)

#每个感兴趣的人选择A B C的购买过程模拟
def buying_A (price,week,inventory):
    sale = 0
    wtp = random.randint(0,WTP[week]+1)
    if wtp >= price[week]:
        if inventory['A'] > 0:
            sale = price[week]
            inventory['A'] += (-1)
        else:
            prob_change = random.random()
            if prob_change <= 0.3:
                if inventory['B'] > 0:
                    sale = price[week]
                    inventory['B'] += (-1)
            elif 0.3 < prob_change <= 0.4:
                if inventory['C'] > 0:
                    sale = price[week]
                    inventory['C'] += (-1)
    revenue[week] += sale


def buying_B (price,week,inventory):
    sale = 0
    wtp = random.randint(0,WTP[week]+1)
    if wtp >= price[week]:
        if inventory['B'] > 0:
            sale = price[week]
            inventory['B'] += (-1)
        else:
            prob_change = random.random()
            if prob_change <= 0.1:
                if inventory['A'] > 0:
                    sale = price[week]
                    inventory['A'] += (-1)
            elif 0.1 < prob_change <= 0.4:
                if inventory['C'] > 0:
                    sale = price[week]
                    inventory['C'] += (-1)
    revenue[week] += sale


def buying_C (price,week,inventory):
    sale = 0
    wtp = random.randint(0,WTP[week]+1)
    if wtp >= price[week]:
        if inventory['C'] > 0:
            sale = price[week]
            inventory['C'] += (-1)
        else:
            prob_change = random.random()
            if prob_change <= 0.1:
                if inventory['B'] > 0:
                    sale = price[week]
                    inventory['B'] += (-1)
    revenue[week] += sale

In [6]:
#大方程，涵盖了前面的部分以及中间选择ABC的过程
def biggest_func(week,price,inventory):
    revenue[week] = 0
    interester = interested(nweek)
    for interest in range(interester):
        interest_index = ((random.randint(1,4)) % 3)
        if interest_index == 0:
            buying_A(price,week,inventory)
        if interest_index == 1:
            buying_B(price,week,inventory)
        if interest_index == 2:
            buying_C(price,week,inventory)
        
    return(inventory,revenue)

In [7]:
def Expand(tree,node):
    if len(tree[node]['Child'])==0:
        for price in price_choice:
            if WTP_0[tree[node]['Week']] < price < WTP[tree[node]['Week']]:
                tree[len(tree)] = {'Week':tree[node]['Week'],'Child':[],'Parent':node,'Price':price,'Type':'S','n':0,'V':0,'UCB':float('inf')}
                tree[node]['Child'].append(len(tree)-1)

In [41]:
def Buildtree(tree,node,k):
    for d in range(k):
    #确定现在的周数和存货量
        notebook = {}
        # Start expansion, selection, and simulation
        #pointer储存现在所在的节点
        pointer = node
        #当所处的周数还没到10周时
        while tree[pointer]['Week'] <= 10:
            # Expansion
            #子节点为0时做expand
            if len(tree[pointer]['Child']) == 0:
                Expand(tree,pointer)
            # Selection
            # 使用WTP的周分布选取本周仿真中会用到的价格
            price_new = []
            for price in price_choice:
                if WTP_0[tree[pointer]['Week']] < price < WTP[tree[pointer]['Week']]:
                    price_new.append(price)
            pointer_0 = tree[pointer]['Child'][0]
            
            for i in range(len(price_new)):
                if tree[tree[pointer]['Child'][i]]['UCB'] > tree[pointer_0]['UCB']:
                    #pointer_0等于的是要选的点
                    pointer_0 = tree[pointer]['Child'][i]
        
            #赋值让现在的点是最大UCB的点
            pointer = pointer_0
            # Simulation
            #得到现在的点的相关信息，为模拟做准备
            week = tree[pointer]['Week']
            price_dic = {week:tree[pointer]['Price']}
            inv = tree[tree[pointer]['Parent']]['State'].copy()
            final = biggest_func(week,price_dic,inv)
            #得到这一周的销售量
            total_revenue = final[1][week]
            #得到的是现有的剩余存货
            total_state = final[0].copy()
            notebook[len(notebook)] = {'Revenue':total_revenue,'State':total_state.copy()}
            #print(notebook)
            #看现在的点有没有子节点
            n_Child = len(tree[pointer]['Child'])
            #当有子节点的时候
            while n_Child > 0:
                #子节点是个D点，更新D点的库存状况，下个循环要用
                if tree[tree[pointer]['Child'][n_Child-1]]['State']==total_state:
                    pointer = tree[pointer]['Child'][n_Child-1]
                    break
                else:
                    n_Child = n_Child-1
            if n_Child == 0:
                tree[len(tree)]={'Week':week+1,'Child':[],'Parent':pointer,'State':total_state.copy(),'Type':'D','n':0,'V':0}
                tree[pointer]['Child'].append(len(tree)-1)
                pointer = len(tree)-1
            #探索现在的子节点中有没有库存相同的点，若有则跳转；反之则创建一个新的子节点
            week = tree[pointer]['Week']
        # Start backpropagation
        #node 是开始的起点，从下往上更新
        BSR = 0
        while pointer > node:
            pointer = tree[pointer]['Parent']
            week = tree[pointer]['Week']
            if tree[pointer]['Type'] == 'S':
                BSR = BSR+notebook[week - tree[node]['Week']]['Revenue']
                rev = (tree[pointer]['V']*tree[pointer]['n']+BSR)/(tree[pointer]['n']+1)
                tree[pointer]['V'] = rev
                tree[pointer]['n'] = tree[pointer]['n']+1 
                #C待更新
                C = 1
                tree[pointer]['UCB'] = tree[pointer]['V']+2*C*(math.log(tree[tree[pointer]['Parent']]['n']+1)/tree[pointer]['n'])**0.5
                sibling_list = tree[tree[pointer]['Parent']]['Child']
                for index in sibling_list:
                    if index != pointer:
                        if tree[index]['n'] != 0:
                            tree[index]['UCB'] = tree[index]['V']+2*C*(math.log(tree[tree[index]['Parent']]['n']+1)/tree[index]['n'])**0.5
            if tree[pointer]['Type'] == 'D':
                tree[pointer]['V'] = (tree[pointer]['V']*tree[pointer]['n']+rev)/(tree[pointer]['n']+1)
                tree[pointer]['n'] = tree[pointer]['n']+1

In [45]:
price_choice = [199,299,399,499,599,699,799,899]
#inventory = {'A':15,'B':15,'C':15}
#初始化状态设置
first_tree = {0:{'Week':7,'Child':[],'State':{'A':4,'B':7,'C':0},'Type':'D','n':0,'V':0}}

start_time = time.time()
Buildtree(first_tree,0,1000)
end_time = time.time()
print(end_time-start_time)
start_time = time.time()
new_tree = {}
for cc in [0,1,2,3]:
    new_tree[cc] = first_tree[cc].copy()
tree_data = pd.DataFrame(data = new_tree)
end_time = time.time()
print(end_time-start_time)
tran_data = tree_data.T
tran_data
#tran_data.to_csv('tree_data_7.csv')

1.1998207569122314
0.001972675323486328


,Week,Child,State,Type,n,V,Parent,Price,UCB
0,7,"[1, 2, 3]","{'A': 4, 'B': 7, 'C': 0}",D,1000,4418.475257,NaN,NaN,NaN
1,7,[4],NaN,S,1,3289.0,0,299,3294.256522
2,7,"[17, 43, 65, 87, 100, 113, 135, 148, 161, 183,...",NaN,S,998,4479.480962,0,399,4479.647354
3,7,[30],NaN,S,1,3289.0,0,499,3294.256522
